<a href="https://colab.research.google.com/github/peeyushsinghal/da/blob/main/mitigating_bias_sa_da_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mitigating bias in sentiment analysis using domain adaptation

In [1]:
! pip install torchtext==0.10.0 --quiet # DOWNGRADE YOUR TORCHTEXT
! pip install ekphrasis --quiet # library to pre process twitter data
! pip install emoji --upgrade --quiet #library to deal with emoji data


     |████████████████████████████████| 7.6 MB 5.0 MB/s 
     |████████████████████████████████| 831.4 MB 2.7 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
     |████████████████████████████████| 83 kB 2.0 MB/s 
     |████████████████████████████████| 45 kB 3.9 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 240 kB 5.3 MB/s 


In [2]:
## Import statements
import pandas as pd
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchtext.legacy.data import Dataset, Field, TabularDataset, BucketIterator
from torchtext.vocab import GloVe
# import torchtext.functional as TTF
import numpy as np
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import emoji
from torchtext.legacy.vocab import Vectors
from tqdm import tqdm
import random
import torch.optim as optim
# import json

In [3]:
# checking device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on:{}".format(DEVICE))

Running on:cuda


## Data loading

In [4]:
#Mounting google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Data Configuration

In [5]:
BASE_PATH = '/content/drive/MyDrive/semeval-2018'
DATA_DIR = os.path.join(BASE_PATH,'datasets')
MODEL_DIR = os.path.join(BASE_PATH,'models')
REF_DIR = os.path.join(BASE_PATH,'reference')

if not os.path.exists(MODEL_DIR):
  os.makedirs(MODEL_DIR)
  print("The new directory is created!")

domain_source = 0.0
domain_target = 1.0

In [6]:
class TASK1(object):
  
    EI_reg = {
        'anger': {
            'train': os.path.join(
                DATA_DIR, 'task1/EI-reg/training/EI-reg-En-anger-train.txt'),
            'dev': os.path.join(
                DATA_DIR, 'task1/EI-reg/development/2018-EI-reg-En-anger-dev.txt'),
            'gold': os.path.join(
                DATA_DIR, 'task1/EI-reg/test-gold/2018-EI-reg-En-anger-test-gold.txt')
                },
        'joy': {
                'train': os.path.join(
                    DATA_DIR, 'task1/EI-reg/training/EI-reg-En-joy-train.txt'),
                'dev': os.path.join(
                    DATA_DIR, 'task1/EI-reg/development/2018-EI-reg-En-joy-dev.txt'),
                'gold': os.path.join(
                    DATA_DIR, 'task1/EI-reg/test-gold/2018-EI-reg-En-joy-test-gold.txt')
                    }
        }

    # V_reg = {
    #     'train': os.path.join(
    #         DATA_DIR, 'task1/V-reg/2018-Valence-reg-En-train.txt'),
    #     'dev': os.path.join(
    #         DATA_DIR, 'task1/V-reg/2018-Valence-reg-En-dev.txt'),
    #     'gold': os.path.join(
    #         DATA_DIR, 'task1/V-reg/2018-Valence-reg-En-test-gold.txt')
    #          }

    EEC = {
        'eec': os.path.join(
            DATA_DIR, 'task1/Equity-Evaluation-Corpus/Equity-Evaluation-Corpus.csv')
             }

## Source Data
Parsing Emotion and Valence regression data : `format [ID	Tweet	Affect Dimension	Intensity Score]`

In [7]:
def parse_reg(data_file, label_format='tuple'):#->  pd.DataFrame:
    """
    This is for datasets for the EI-reg and V-reg English tasks 
    Returns:
        df: dataframe with columns in the first row of file [ID-Tweet-Affect Dimension-Intensity Score]
    """
    with open(data_file, 'r') as fd:
      data = [l.strip().split('\t') for l in fd.readlines()]
    
    # print(data)
    df = pd.DataFrame (data[1:],columns=data[0])
    csv_file_name = (data_file.split("/")[-1]).split('.')[0]+".csv"
    csv_file = df.to_csv(str(csv_file_name))
    # df['domain'] = domain_source
    return csv_file_name
    # return df

In [ ]:
parse_reg(os.path.join(DATA_DIR, 'task1/EI-reg/training/EI-reg-En-joy-train.txt'))

## Source Parser
Generic Source Data Parser

In [8]:
def parse_csv(task, dataset, emotion='anger'):
    if task == 'EI-reg':
        data_train = TASK1.EI_reg[emotion][dataset]
        csv_file_name = parse_reg(data_train)
        return csv_file_name
        # df[df.columns[-1]] = df[df.columns[-1]].astype(float)
        # return df
    # elif task == 'V-reg':
    #     data_train = TASK1.V_reg[dataset]
    #     df = parse_reg(data_train)
    #     df[df.columns[-1]] = df[df.columns[-1]].astype(float)
    #     return df
    else:
        return None

In [9]:
file_EI_reg_train = parse_csv('EI-reg','train','joy')
file_EI_reg_train

file_EI_reg_val = parse_csv('EI-reg','dev','joy')
print(file_EI_reg_val)

file_EI_reg_test = parse_csv('EI-reg','gold','joy')
(file_EI_reg_test)

2018-EI-reg-En-joy-dev.csv


'2018-EI-reg-En-joy-test-gold.csv'

## Preprocess tweets

In [10]:
# reference : https://github.com/cbaziotis/ekphrasis


text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [ ]:
# #### Example checks of pre-processing
# sentences = [
#     "CANT WAIT for the new season of #TwinPeaks ＼(^o^)／!!! #davidlynch #tvseries :)))",
#     "I saw the new #johndoe movie and it suuuuucks!!! WAISTED $10... #badmovies :/",
#     "@SentimentSymp:  can't wait for the Nov 9 #Sentiment talks!  YAAAAAAY !!! :-D http://sentimentsymposium.com/.",
#     "@MGBarbieri @SpalkTalk a@b.com And just saw your LinkedIn comment after I sent this! Thanks for the message :) 😀",
#     "💙💛🏆 @GeorgeePitman Young Player of The Season 🏆💛💙 #irony #actuallyseventy"
# ]

# for s in sentences:
#     print(" ".join(text_processor.pre_process_doc(s)))
# # print ([text_processor.pre_process_doc(s) for s in sentences])

In [11]:
def preprocess_tweet(tweet): 
  tweet_processed = text_processor.pre_process_doc(tweet)
  # print (tweet_processed)
  final_list =[]
  for index, tweet in enumerate(tweet_processed):
      final_list.append(emoji.demojize(tweet, language = 'en'))
  
  # print(df)
  return final_list

In [ ]:
# #### Example checks of pre-processing
# sentences = [
#     "CANT WAIT for the new season of #TwinPeaks ＼(^o^)／!!! #davidlynch #tvseries :)))",
#     "I saw the new #johndoe movie and it suuuuucks!!! WAISTED $10... #badmovies :/",
#     "@SentimentSymp:  can't wait for the Nov 9 #Sentiment talks!  YAAAAAAY !!! :-D http://sentimentsymposium.com/.",
#     "@MGBarbieri @SpalkTalk a@b.com And just saw your LinkedIn comment after I sent this! Thanks for the message :) 😀",
#     "💙💛🏆 @GeorgeePitman Young Player of The Season 🏆💛💙 #irony #actuallyseventy"
# ]

# for s in sentences:
#   print(preprocess_tweet(s))
#   # print(" ".join(preprocess_tweet(s)))

## TorchText Treatment

In [18]:
MAX_SIZE = 50
field_tweet = Field(sequential=True, use_vocab = True, tokenize = preprocess_tweet, fix_length = MAX_SIZE, batch_first = True)
# field_tweet = Field(sequential=True, use_vocab = False, tokenize = preprocess_tweet)

field_intensity = Field(sequential= False, 
                        dtype = torch.float,
                        use_vocab = False 
                        )

In [19]:
fields = {
    'Tweet':('tweet', field_tweet ), #
    'Intensity Score': ('intensity',field_intensity) # Intensity Score is name of the dataset column, field_intensity is how we have defined the field, intensity is the name of the variable going fwd
    }

In [20]:
train_data, valid_data, test_data = TabularDataset.splits( path = './', 
                                              train = file_EI_reg_train,
                                              validation = file_EI_reg_val, 
                                              test = file_EI_reg_test,
                                              format = 'csv',
                                              fields = fields
                                          )

In [ ]:
print(train_data[0].__dict__.keys())

In [ ]:
print(valid_data[0].__dict__.values())

In [ ]:
print((train_data[2].__dict__.values()))

In [ ]:
count = 0
for example in test_data.examples:
  print(example.tweet, example.intensity)
  count += 1
  if count > 10:
    break

## Building iterator and Vocabulary

In [25]:
MAX_VOCAB_SIZE = 10000

field_tweet.build_vocab(train_data, 
                  max_size = MAX_VOCAB_SIZE,
                  min_freq = 1,
                  vectors = "glove.6B.100d",
                  unk_init=torch.Tensor.normal_)

vocab = field_tweet.build_vocab(train_data, 
                  max_size = MAX_VOCAB_SIZE,
                  min_freq = 1,
                  vectors = "glove.6B.100d",
                  unk_init=torch.Tensor.normal_)
field_intensity.build_vocab(train_data)

In [ ]:
type(vocab)

In [24]:
BATCH_SIZE = 2
train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), 
                                                      batch_size= BATCH_SIZE,
                                                      sort_key = lambda x: len(x.tweet),
                                                      sort_within_batch=True,
                                                      device = DEVICE,
                                                      shuffle= True)

In [17]:
count = 0
for batch in train_iterator:
  print (batch.tweet)
  print (batch.intensity)
  count += 1
  if count > 2:
    break

tensor([[  39,    3,   53,    2,   15,   10, 1822,   16,   39,   58,   45,    4,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1],
        [   5,   21, 1084,   20,    6,  132,   16,   39,  506,   15,   60,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1]], device='cuda:0')
tensor([0.6040, 0.6250], device='cuda:0')
tensor([[   5,  273,   21,  135,    4,   41,  195,  110,  164,   71, 2006,  271,
          216,    4,   18,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
     

In [ ]:
print(list(field_tweet.vocab.stoi.items()))

In [ ]:
print(field_tweet.vocab.vectors[field_tweet.vocab.stoi['the']])

In [ ]:
field_tweet.vocab.vectors[9]

In [ ]:
# for batch_number, batch in enumerate(train_iterator):
#   print(batch_number)
#   print(batch.tweet)
#   print(batch.intensity)

## CNN 1d model

### Gradient Reversal layer

In [26]:
from torch.autograd import Function

class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

CNN 1 D model
Reference: A Sensitivity Analysis of (and Practitioners' Guide to) Convolutional Neural Networks for Sentence Classification, Ye Zhang, Byron Wallace 2015

Difference:

use of embedding
use of sigmoid function, as we are having a regression model not a classififer as the main task

In [22]:
import torch.nn as nn
import torch.nn.functional as F

class CNN1d(nn.Module):
    def __init__(self, 
                 vocab_size, 
                 embedding_dim, 
                 n_filters, 
                 filter_sizes, 
                 output_dim, 
                 dropout, 
                 pad_idx
                 ):
        super().__init__()
        
        #---------------------Feature Extractor Network----------------------#
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)

        # Convolutional Network
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        
        #---------------------Regression Network------------------------#
        # Fully-connected layer and Dropout
        self.regression = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(len(filter_sizes) * n_filters, len(filter_sizes) * n_filters // 2),
            nn.ReLU(),
            nn.Linear(len(filter_sizes) * n_filters // 2, output_dim * 10),
            nn.ReLU(),
            nn.Linear(output_dim * 10, output_dim)
            # ,
            # nn.Sigmoid()
        )
        # self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim * 10)
        # self.fc2 = nn.Linear(output_dim * 10, output_dim)
        # self.dropout = nn.Dropout(dropout)

        #---------------------Domain Classifier Network------------------------#
        # Fully-connected layer and Dropout
        self.domain_classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(len(filter_sizes) * n_filters, len(filter_sizes) * n_filters // 2),
            nn.ReLU(),
            nn.Linear(len(filter_sizes) * n_filters // 2, output_dim * 10),
            nn.ReLU(),
            nn.Linear(output_dim * 10, output_dim)
        )
        
    def forward(self, text, alpha=1):
        
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.permute(0, 2, 1)
        
        #embedded = [batch size, emb dim, sent len]
        
        conved = [F.relu(conv(embedded)) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        x_feature = torch.cat(pooled, dim = 1)
        
        #x_feature = [batch size, n_filters * len(filter_sizes)]
        
        reverse_feature = ReverseLayerF.apply(x_feature, alpha)
        # print("reverse_feature",reverse_feature)
    
        regression_output = self.regression(x_feature)
    
        domain_classifier_output = self.domain_classifier(reverse_feature)


        return regression_output, domain_classifier_output

In [28]:
INPUT_DIM = len(field_tweet.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2, 3, 4, 5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = field_tweet.vocab.stoi[field_tweet.pad_token]

In [29]:
model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
model.to(DEVICE)

CNN1d(
  (embedding): Embedding(4788, 100, padding_idx=1)
  (convs): ModuleList(
    (0): Conv1d(100, 100, kernel_size=(2,), stride=(1,))
    (1): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
    (2): Conv1d(100, 100, kernel_size=(4,), stride=(1,))
    (3): Conv1d(100, 100, kernel_size=(5,), stride=(1,))
  )
  (regression): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=400, out_features=200, bias=True)
    (2): ReLU()
    (3): Linear(in_features=200, out_features=10, bias=True)
    (4): ReLU()
    (5): Linear(in_features=10, out_features=1, bias=True)
  )
  (domain_classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=400, out_features=200, bias=True)
    (2): ReLU()
    (3): Linear(in_features=200, out_features=10, bias=True)
    (4): ReLU()
    (5): Linear(in_features=10, out_features=1, bias=True)
  )
)

### Load Pre trained embeddings
we'll load the pre-trained *embeddings*

In [34]:
pretrained_embeddings = field_tweet.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.6223,  0.7095, -0.4468,  ..., -1.6087,  0.5500,  0.1419],
        [-0.5598, -0.1254, -0.4154,  ...,  0.8011, -0.8856, -0.0770],
        [ 1.0338, -0.1840,  0.1118,  ..., -0.0783,  1.2427, -0.5635],
        ...,
        [-0.3344,  2.4452,  1.7789,  ...,  0.9308,  0.2818, -1.3562],
        [-0.8445, -0.1172,  0.2783,  ..., -1.5594,  1.1036, -0.1830],
        [-1.9549, -0.7499,  0.3245,  ..., -0.1048, -1.0951, -2.0031]],
       device='cuda:0')

In [35]:
field_tweet.vocab.vectors.shape

torch.Size([4788, 100])

In [36]:
UNK_IDX = field_tweet.vocab.stoi[field_tweet.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## Training the model

### Without training one forward pass

In [60]:
for batch in train_iterator:
  print(batch.tweet)
  output = model(batch.tweet)
  print (output)
  break

tensor([[   6,  734,  135,    9, 4121, 3848,    3,   31,    2,   15,    9,   47,
          118,  249,   12,  206,   20,   17,  118,  714,    4, 3082, 1510, 3670,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1],
        [   5,    5,    5, 2525, 2583,   15,  102,  130,  202,    4,  259, 1007,
          273,   61,  171,    4,   70,  141, 2626,  171, 2464,   33,  463,  154,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1]], device='cuda:0')
(tensor([[0.4910],
        [0.3248]], device='cuda:0', grad_fn=<AddmmBackward>), tensor([[0.0985],
        [0.0635]], device='cuda:0', grad_fn=<AddmmBackward>))


In [38]:
# import torch.optim as optim

# optimizer = optim.Adam(model.parameters())

# criterion = nn.BCEWithLogitsLoss()

# model = model.to(DEVICE)
# criterion = criterion.to(DEVICE)

### Typical Train Model Function

In [39]:
# Typical Training Function

from tqdm import tqdm # for beautiful model training updates


def train_model(model, device, train_loader, optimizer, epoch):
    model.train() # setting the model in training mode
    pbar = tqdm(train_loader) # putting the iterator in pbara
    correct = 0 # for accuracy numerator
    processed =0 # for accuracy denominator
    epoch_loss = 0.0
    for batch_idx, batch in enumerate(pbar):

        tweets, intensities = batch.tweet.to(device), batch.intensity.to(device)  # plural, we are not interested in domain
        #sending data to CPU or GPU as per device

        optimizer.zero_grad() # setting gradients to zero to avoid accumulation

        y_preds,_ = model(tweets) # forward pass, result captured in y_preds (plural as there are many body in a batch)
        # we are not interested in domain prediction
        # the predictions are in one hot vector

        regression_loss = regression_loss_function(y_preds,intensities.unsqueeze(1)) # Computing loss
        # loss = F.mse_loss(y_preds,intensities.unsqueeze(1)) # Computing loss

        train_regresion_losses.append(regression_loss.item()) # to capture loss over many epochs

        regression_loss.backward() # backpropagation
        optimizer.step() # updating the params

        # preds = y_preds.argmax(dim=1, keepdim=True)  # get the index olf the max log-probability
        # correct += preds.eq(labels.view_as(preds)).sum().item()
        epoch_loss += regression_loss.item()

        processed += len(tweets)

        pbar.set_description(desc= f'Loss={regression_loss.item()} Batch_id={batch_idx} Epoch Average loss={100*epoch_loss/processed:0.4f}')
    train_accuracy.append(100*epoch_loss/len(train_loader))

### Typical Test Function

In [40]:
def test_model(model,device, data_loader, mode= 'test'):
    model.eval() # setting the model in evaluation mode
    loss = 0
    correct = 0 # for accuracy numerator
    test_regresion_losses =[] # for overall batches (summed over batches)
    valid_regresion_losses =[] # for overall batches (summed over batches)

    with torch.no_grad():
        for batch in data_loader:

            tweets, intensities  = batch.tweet.to(device), batch.intensity.to(device) #sending data to CPU or GPU as per device
            # we are not interested in domains
            
            y_preds,_ = model(tweets) # forward pass, result captured in outputs (plural as there are many bodies in a batch)
            # the outputs are in batch size x one hot vector 
            # not interested in domain output

            regression_loss = regression_loss_function(y_preds,intensities.unsqueeze(1))

            if mode == 'test':
              test_regresion_losses.append(regression_loss.item())
              # print(f'...in the batch...{regression_loss}')
            else:
              valid_regresion_losses.append(regression_loss.item())
              # print(f'...in the batch...{regression_loss}')

        # regression_loss.item() /= len(data_loader.dataset) # average test loss
        if mode == 'test':
          # total_test_regression_loss = sum(test_regresion_losses)
          # test_regresion_losses.append(regression_loss) # to capture loss over many batches
          # print('...Average test loss: {:.8f}'.format((total_test_regression_loss)/len(data_loader.dataset)))
          print(f'...Average batch test loss is {sum(test_regresion_losses) / len(data_loader)}')
        else:
          # valid_regresion_losses.append(regression_loss) # to capture loss over many batches
          # total_valid_regression_loss = sum(valid_regresion_losses)
          # print('...Average validation loss: {:.8f}'.format((total_valid_regression_loss)/len(data_loader.dataset)))
          print(f'...Average batch validation loss is {sum(valid_regresion_losses) / len(data_loader)}')

In [41]:
# EXECUTION


optimizer = optim.Adam(model.parameters(), lr=0.001)
domain_loss_function= nn.BCEWithLogitsLoss()
regression_loss_function = nn.L1Loss()


model = model.to(DEVICE)
domain_loss_function = domain_loss_function.to(DEVICE)
regression_loss_function = regression_loss_function.to(DEVICE)

# train_losses = [] # to capture train losses over training epochs
train_accuracy = [] # to capture train accuracy over training epochs
# val_losses = [] # to capture validation loss
# test_losses = [] # to capture test losses 
# test_accuracy = [] # to capture test accuracy 

# EPOCHS = 2
EPOCHS = 100
# dict_val_loss = {}
# dict_test_loss = {}


train_regresion_losses = [] # to capture train losses over training epochs
train_domain_losses = []
train_accuracy = [] # to capture train accuracy over training epochs
# valid_regresion_losses = [] # to capture validation loss
# test_regresion_losses = [] # to capture test losses 
total_test_regression_loss =[]
total_valid_regression_loss =[]
# print(f'----------------------training started for {name}-----------------')
for epoch in range(EPOCHS):
  print("EPOCH:", epoch+1)
  train_model(model, DEVICE, train_iterator, optimizer, epoch)
  print("for validation.......")
  # val_name = train_name.replace("train", "val" )
  # test_model(typical_model, device, dict_val_loader[val_name], mode = 'val')
  test_model(model, DEVICE, valid_iterator, mode = 'val')


  print("for test  .......")
  # test_name = train_name.replace("train", "test" )
  # test_model(typical_model, device, dict_test_loader[test_name], mode = 'test')
  test_model(model, DEVICE, test_iterator, mode = 'test')

# dict_val_loss[name] = val_losses
# dict_test_loss[name] = test_losses

model_name = "Non_DANN"+".pt"
torch.save(model.state_dict(), os.path.join(MODEL_DIR, model_name))
# print(f'----------------------training complete for {name}-----------------')
# print(dict_val_loss.items())
# print(dict_test_loss.items())

EPOCH: 1


Loss=0.32847732305526733 Batch_id=807 Epoch Average loss=10.4675: 100%|██████████| 808/808 [00:06<00:00, 127.01it/s]


for validation.......
...Average batch validation loss is 0.15835808782741942
for test  .......
...Average batch test loss is 0.1604405780866823
EPOCH: 2


Loss=0.3056570887565613 Batch_id=807 Epoch Average loss=8.3242: 100%|██████████| 808/808 [00:06<00:00, 128.66it/s]


for validation.......
...Average batch validation loss is 0.1376844279724976
for test  .......
...Average batch test loss is 0.13053059245158277
EPOCH: 3


...

for validation.......
...Average batch validation loss is 0.1388239760337205
for test  .......
...Average batch test loss is 0.1314581407345225
EPOCH: 99


Loss=0.017631642520427704 Batch_id=807 Epoch Average loss=1.4902: 100%|██████████| 808/808 [00:07<00:00, 111.85it/s]


for validation.......
...Average batch validation loss is 0.13302243619129575
for test  .......
...Average batch test loss is 0.12477949513610836
EPOCH: 100


Loss=0.018109366297721863 Batch_id=807 Epoch Average loss=1.6461: 100%|██████████| 808/808 [00:07<00:00, 114.00it/s]


for validation.......
...Average batch validation loss is 0.13367241785444062
for test  .......
...Average batch test loss is 0.12546751397589448


# Equality Evaluation using Equity Evaluation Corpus

## Loading Evaluation Data
format `[ID	Sentence	Template	Person	Gender	Race Emotion	Emotion word]`

In [62]:
data_EEC = TASK1.EEC['eec']
df_EEC = pd.read_csv(data_EEC)

In [85]:
df_EEC.head()

,ID,Sentence,Template,Person,Gender,Race,Emotion,Emotion word
0,2018-En-mystery-05498,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,angry
1,2018-En-mystery-11722,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,furious
2,2018-En-mystery-11364,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,irritated
3,2018-En-mystery-14320,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,enraged
4,2018-En-mystery-14114,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,annoyed


## Creating evalution function (includes pre-processing)

In [55]:
## padding function : adds padding / truncates to max size
def pad_or_truncate(some_list, target_len = MAX_SIZE ,pad_idx = PAD_IDX):
    return some_list[:target_len] + [pad_idx]*(target_len - len(some_list))

## preprocessing function, takes in a tweet and returns padded indexed tweet (input for model)
def text_pipeline(tweet):
    indexed_tweet = [field_tweet.vocab.__getitem__(token) for token in preprocess_tweet(tweet)]
    # print(indexed_tweet)
    return pad_or_truncate(indexed_tweet, MAX_SIZE , pad_idx = PAD_IDX)
    # print(indexed_tweet_padded)

In [56]:
# i = random.randint(0,len(df_EEC))
# tweet_example = df_EEC['Sentence'][i]
# print(tweet_example, text_pipeline(tweet_example))

## Loading model

In [57]:
### Loading Model
model_name = "Non_DANN.pt"
loaded_model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
loaded_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, model_name)))
loaded_model.eval()

CNN1d(
  (embedding): Embedding(4788, 100, padding_idx=1)
  (convs): ModuleList(
    (0): Conv1d(100, 100, kernel_size=(2,), stride=(1,))
    (1): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
    (2): Conv1d(100, 100, kernel_size=(4,), stride=(1,))
    (3): Conv1d(100, 100, kernel_size=(5,), stride=(1,))
  )
  (regression): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=400, out_features=200, bias=True)
    (2): ReLU()
    (3): Linear(in_features=200, out_features=10, bias=True)
    (4): ReLU()
    (5): Linear(in_features=10, out_features=1, bias=True)
  )
  (domain_classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=400, out_features=200, bias=True)
    (2): ReLU()
    (3): Linear(in_features=200, out_features=10, bias=True)
    (4): ReLU()
    (5): Linear(in_features=10, out_features=1, bias=True)
  )
)

In [58]:
def predict(tweet,model, text_pipeline):
  with torch.no_grad():
    tweet_tensor = torch.tensor(text_pipeline(tweet)).unsqueeze(0).to(DEVICE)
    # output = None
    # print(tweet_tensor.shape)
    output = model(tweet_tensor)
    return output[0].item()

In [65]:
# i = random.randint(0,len(df_EEC))
# tweet_example = df_EEC['Sentence'][i]
# model = loaded_model.to(DEVICE)
# print(predict(tweet_example, loaded_model,text_pipeline))

0.40531787276268005


## Creating Sentence pairs (as per SEMVAL18 paper)

In [69]:
dict_f_m_noun_phrase = {'she':'he', 
            'her':'him',
            'this woman':'this man',
            'this girl':'this boy',
            'my sister' : 'my brother',
            'my daughter' : 'my son',
            'my wife': 'my husband',
            'my girlfriend':'my boyfriend',
            'my mother':'my father',
            'my aunt':'my uncle',
            'my mom': 'my dad'
            }

name_male = ['Alonzo','Jamel','Alphonse','Jerome','Leroy','Torrance','Darnell','Lamar','Malik','Terrence','Adam','Harry','Josh','Roger','Alan','Frank','Justin','Ryan','Andrew','Jack'] 
name_female = ['Nichelle','Shereen','Ebony','Latisha','Shaniqua','Jasmine','Tanisha','Tia','Lakisha','Latoya','Amanda','Courtney','Heather','Melanie','Katie','Betsy','Kristin','Nancy','Stephanie','Ellen']


In [132]:
list_unique_template = list(df_EEC['Template'].dropna().unique())
# print(list_unique_template)
list_emotion_word = list(df_EEC['Emotion word'].unique())
# print(list_emotion_word)
list_gender = list(df_EEC['Gender'].dropna().unique())
# print(list_gender)
list_person = list(df_EEC['Person'].unique())   
# print(list_person)

In [82]:
# list_f_m_noun_phrase =[]
# list_f_m_noun_phrase.extend(name_male)
# list_f_m_noun_phrase.extend(name_female)
# [list_f_m_noun_phrase.extend([f,m]) for f,m in dict_f_m_noun_phrase.items()]
# print(list_f_m_noun_phrase)
# assert set(list_f_m_noun_phrase)<= set(list_person), "The noun phrases are not subset of overall person list"

['Alonzo', 'Jamel', 'Alphonse', 'Jerome', 'Leroy', 'Torrance', 'Darnell', 'Lamar', 'Malik', 'Terrence', 'Adam', 'Harry', 'Josh', 'Roger', 'Alan', 'Frank', 'Justin', 'Ryan', 'Andrew', 'Jack', 'Nichelle', 'Shereen', 'Ebony', 'Latisha', 'Shaniqua', 'Jasmine', 'Tanisha', 'Tia', 'Lakisha', 'Latoya', 'Amanda', 'Courtney', 'Heather', 'Melanie', 'Katie', 'Betsy', 'Kristin', 'Nancy', 'Stephanie', 'Ellen', 'she', 'he', 'her', 'him', 'this woman', 'this man', 'this girl', 'this boy', 'my sister', 'my brother', 'my daughter', 'my son', 'my wife', 'my husband', 'my girlfriend', 'my boyfriend', 'my mother', 'my father', 'my aunt', 'my uncle', 'my mom', 'my dad']


In [133]:
print(list_emotion_word)
# list_emotion_word= list_emotion_word.append('')
# print(list_emotion_word)

['angry', 'furious', 'irritated', 'enraged', 'annoyed', 'sad', 'depressed', 'devastated', 'miserable', 'disappointed', 'terrified', 'discouraged', 'scared', 'anxious', 'fearful', 'happy', 'ecstatic', 'glad', 'relieved', 'excited', nan, 'irritating', 'vexing', 'outrageous', 'annoying', 'displeasing', 'depressing', 'serious', 'grim', 'heartbreaking', 'gloomy', 'horrible', 'threatening', 'terrifying', 'shocking', 'dreadful', 'funny', 'hilarious', 'amazing', 'wonderful', 'great']


In [139]:
# Template - F - M Noun Phrases chunks
dict_sentence_pair = {}

for template in list_unique_template:
  for f, m in dict_f_m_noun_phrase.items():
    condition_1 = df_EEC['Template']== template
    condition_2 = df_EEC['Person']== f
    condition_3 = df_EEC['Person']== m
    df_temp_f = df_EEC[(condition_1 & condition_2 )] 
    df_temp_m = df_EEC[(condition_1 & condition_3 )]
    # count = 0
    for emotion_word in list_emotion_word:
      
      condition_4 = df_EEC['Emotion word'] == emotion_word
      
      k = df_temp_f[condition_4]['Sentence']
      v = df_temp_m[condition_4]['Sentence']
      if len(k) > 0 and len (v) > 0:
        dict_sentence_pair[k.values[0]] = v.values[0]
      
      ## Checking for column values where emotion word value blank
      if pd.isnull(emotion_word):
        k_null = df_temp_f[df_temp_f['Emotion word'].isna()]['Sentence']
        v_null = df_temp_m[df_temp_m['Emotion word'].isna()]['Sentence']
        if len(k_null) > 0 and len (v_null) > 0:
          dict_sentence_pair[k_null.values[0]] = v_null.values[0]
      
print(len(dict_sentence_pair),"\ndict_sentence_pair............ \n",dict_sentence_pair)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


1440 
dict_sentence_pair............ 
 {'She feels angry.': 'He feels angry.', 'She feels furious.': 'He feels furious.', 'She feels irritated.': 'He feels irritated.', 'She feels enraged.': 'He feels enraged.', 'She feels annoyed.': 'He feels annoyed.', 'She feels sad.': 'He feels sad.', 'She feels depressed.': 'He feels depressed.', 'She feels devastated.': 'He feels devastated.', 'She feels miserable.': 'He feels miserable.', 'She feels disappointed.': 'He feels disappointed.', 'She feels terrified.': 'He feels terrified.', 'She feels discouraged.': 'He feels discouraged.', 'She feels scared.': 'He feels scared.', 'She feels anxious.': 'He feels anxious.', 'She feels fearful.': 'He feels fearful.', 'She feels happy.': 'He feels happy.', 'She feels ecstatic.': 'He feels ecstatic.', 'She feels glad.': 'He feels glad.', 'She feels relieved.': 'He feels relieved.', 'She feels excited.': 'He feels excited.', 'This woman feels angry.': 'This man feels angry.', 'This woman feels furious.':